# Token Based Indexing

When indexing hidden states for specific tokens, use `.token[<idx>]` or `.t[<idx>]`.

As a preliminary example, lets just get a hidden state from the model using `.t[<idx>]`.

In [1]:
from nnsight import LanguageModel

model = LanguageModel('openai-community/gpt2', device_map='cuda')

In [2]:
with model.trace('The Eiffel Tower is in the city of') as tracer:

    hidden_states = model.transformer.h[-1].output[0].t[0].save()
    output = model.output.save()

print(hidden_states.shape)
print(output.shape)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([1, 768])
CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ -36.2874,  -35.0114,  -38.0793,  ...,  -40.5163,  -41.3759,
           -34.9193],
         [ -68.8886,  -70.1562,  -71.8408,  ...,  -80.4195,  -78.2552,
           -71.1206],
         [ -82.2950,  -81.6519,  -83.9941,  ...,  -94.4878,  -94.5194,
           -85.6998],
         ...,
         [-113.8675, -111.8628, -113.6634,  ..., -116.7652, -114.8267,
          -112.3621],
         [ -81.8531,  -83.3006,  -91.8192,  ...,  -92.9943,  -89.8382,
           -85.6897],
         [-103.9307, -102.5054, -105.1563,  ..., -109.3099, -110.4195,
          -103.1395]]], device='cuda:0', grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-0.9420,  1.9023,  0.8722,  ..., -1.2703, -0.4792,  1.2469],
          [-1.9590,  2.7141,  2.8423,  ..., -1.1633, -1.6173,  2.1507],
          [-2.6123,  2.0937,  0.9679,  ..., -0.9763, -1.2243,  2.0279],
          ...,
          [-2.4282,  2.4462,  2.1550,  ..., -0.5916, -1

Lets see why token based indexing is necessary.

In this example, we call invokes on two inputs of different tokenized length. We **incorrectly** index into the hidden states using normal python indexing.

In [3]:
from rich import print

with model.trace() as tracer:
    with tracer.invoke('The') as invoker:
        incorrect_a =  model.transformer.input[0][0][:,0].save()
        
    with tracer.invoke('The Eiffel Tower is in the city of''The Eiffel Tower is in the city of') as invoker:
        incorrect_b = model.transformer.input[0][0][:,0].save()

print(f"Shorter input: {incorrect_a.value}")
print(f"Longer input: {incorrect_b.value}")

Shorter input: tensor([50256], device='cuda:0')

Longer input: tensor([464], device='cuda:0')

Notice how we indexed into the first token for both strings but recieved a different result from each invoke. **This is because if there are multiple invocations, padding is performed on the left side so these helper functions index from the back.**

Let's correctly index into the hidden states using token based indexing.

In [4]:
with model.trace() as tracer:
    with tracer.invoke('The') as invoker:
        correct_a =  model.transformer.input[0][0].t[0].save()
        
    with tracer.invoke('The Eiffel Tower is in the city of') as invoker:
        correct_b = model.transformer.input[0][0].t[0].save()

print(f"Shorter input: {correct_a.value}")
print(f"Longer input: {correct_b.value}")

Shorter input: tensor([464], device='cuda:0')

Longer input: tensor([464], device='cuda:0')

Now we have the correct tokens!